## 四跨连续箱梁桥 OpenSeesPy 建模 Notebook

本 Notebook 聚焦于四跨四柱式连续箱梁桥的地震响应建模。研究对象遵循现行中国公路桥梁抗震设计规范的构造要求，采用 OpenSeesPy 的三维框架单元表达上部结构与下部墩台。模型计算域限定为整桥结构体系，自由度维数对应桥面与墩柱节点的三向平动与扭转自由度，并考虑台后回填土、支座、剪力键与碰撞缝的附属非线性。以下单元格依次建立几何、材料、截面、零长度单元、基础柔度、上部结构、部件函数、整桥组装与地震分析流程。

In [ ]:
# 参数面板：集中定义几何、材料、支承与分析参数
# 引用：OpenSeesPy 文档 https://openseespydoc.readthedocs.io
params = {
    "units": {
        "length": "m",
        "force": "kN",
        "mass": "ton"
    },
    "bridge": {
        "span_lengths": [40.0, 45.0, 45.0, 40.0],  # 四跨跨长
        "pier_coordinates": [
            (0.0, 0.0, 0.0),
            (40.0, 0.0, 0.0),
            (85.0, 0.0, 0.0),
            (130.0, 0.0, 0.0),
            (170.0, 0.0, 0.0)
        ],
        "deck_elevation": 15.0,
        "pier_heights": [14.0, 15.0, 15.0, 14.0]
    },
    "section": {
        "deck": {
            "width": 12.5,
            "depth": 3.0,
            "web_thickness": 0.25,
            "flange_thickness": 0.18,
            "num_fibers": 32
        },
        "pier": {
            "diameter": 2.2,
            "cover": 0.05,
            "longitudinal_bars": 20,
            "bar_area": 0.000314,
            "transverse_spacing": 0.2
        }
    },
    "materials": {
        "concrete_cover": {
            "fc": -35.0,
            "epsc0": -0.002,
            "fcu": -20.0,
            "epscu": -0.01,
            "lambda": 0.1,
            "ft": 3.5,
            "Ets": 1200.0
        },
        "concrete_core": {
            "fc": -40.0,
            "epsc0": -0.0025,
            "fcu": -25.0,
            "epscu": -0.015,
            "lambda": 0.1,
            "ft": 4.0,
            "Ets": 1500.0
        },
        "steel": {
            "fy": 500.0,
            "E0": 200000.0,
            "b": 0.01,
            "R0": 18.0,
            "cR1": 0.925,
            "cR2": 0.15
        },
        "bond": {
            "s1": 0.004,
            "tau1": 8.0,
            "s2": 0.012,
            "tau2": 6.0,
            "s3": 0.04,
            "tau3": 2.0,
            "K": 1200.0,
            "r": 0.5,
            "c": 1.0,
            "phi": 1.0,
            "alpha": 0.3,
            "beta": 0.0
        }
    },
    "foundation": {
        "translational_stiffness": [60000.0, 60000.0, 80000.0],
        "rotational_stiffness": [25000.0, 25000.0, 40000.0]
    },
    "abutment": {
        "backfill_k": 40000.0,
        "backfill_uy": 0.08,
        "residual_ratio": 0.2
    },
    "bearing": {
        "ke": 18000.0,
        "yield_force": 350.0,
        "kv": 100000.0,
        "eta": 0.05
    },
    "shear_key": {
        "k1": 32000.0,
        "k2": 8000.0,
        "k3": 2000.0,
        "fy1": 300.0,
        "fy2": 450.0,
        "residual": 0.3
    },
    "impact": {
        "gap": 0.05,
        "k1": 150000.0,
        "k2": 5000.0,
        "mu": 0.65
    },
    "analysis": {
        "gravity_steps": 10,
        "dt": 0.005,
        "time_series_file": "input/ground_motion.txt",
        "record_dt": 0.01,
        "total_time": 20.0
    }
}


### 建立 OpenSeesPy 域并设定单位
在此格中初始化 OpenSeesPy 域，声明采用三维框架单元，载入可视化模块 `opsvis` 并固定单位制。

In [ ]:
import openseespy.opensees as ops
import opsvis as opsv

ops.wipe()
ops.model('basic', '-ndm', 3, '-ndf', 6)

units = params["units"]
print(f"Units: length={units['length']}, force={units['force']}, mass={units['mass']}")


### 几何参数化与节点布置
沿桥梁中心线生成节点坐标，并根据跨长自动划分。

In [ ]:
import numpy as np

span_lengths = params["bridge"]["span_lengths"]
pier_coords = params["bridge"]["pier_coordinates"]
deck_elev = params["bridge"]["deck_elevation"]

node_tags = []
current_x = 0.0
node_tag = 1
for span in span_lengths:
    num_div = 4
    for i in range(num_div):
        x = current_x + span * i / num_div
        ops.node(node_tag, x, 0.0, deck_elev)
        node_tags.append(node_tag)
        node_tag += 1
    current_x += span
ops.node(node_tag, current_x, 0.0, deck_elev)
node_tags.append(node_tag)

# 墩顶节点
pier_top_tags = []
for idx, coord in enumerate(pier_coords):
    tag = 100 + idx
    height = params["bridge"]["pier_heights"][idx] if idx < len(params["bridge"]["pier_heights"]) else deck_elev
    ops.node(tag, coord[0], coord[1], coord[2] + height)
    pier_top_tags.append(tag)

print(f"Created {len(node_tags)} deck nodes and {len(pier_top_tags)} pier-top nodes")


### 材料定义
使用 Concrete02 与 Steel02 描述混凝土与钢筋，参数注释说明物理意义与来源。

In [ ]:
mat_id = 1
conc_cover = params["materials"]["concrete_cover"]
conc_core = params["materials"]["concrete_core"]
steel = params["materials"]["steel"]
# Concrete02 参数含义：fc=抗压强度，epsc0=峰值压应变，fcu=极限压碎应力，epscu=极限压碎应变，
# lambda=卸载刚度因子，ft=抗拉强度，Ets=张拉软化斜率；参见 OpenSeesPy Concrete02 文档。
ops.uniaxialMaterial('Concrete02', mat_id, conc_cover['fc'], conc_cover['epsc0'], conc_cover['fcu'], conc_cover['epscu'], conc_cover['lambda'], conc_cover['ft'], conc_cover['Ets'])
cover_mat = mat_id
mat_id += 1
ops.uniaxialMaterial('Concrete02', mat_id, conc_core['fc'], conc_core['epsc0'], conc_core['fcu'], conc_core['epscu'], conc_core['lambda'], conc_core['ft'], conc_core['Ets'])
core_mat = mat_id
mat_id += 1
# Steel02 参数：fy=屈服强度，E0=初始弹模，b=硬化系数，R0/cR1/cR2=Baushinger 参数；参见 OpenSeesPy Steel02 文档。
ops.uniaxialMaterial('Steel02', mat_id, steel['fy'], steel['E0'], steel['b'], steel['R0'], steel['cR1'], steel['cR2'])
steel_mat = mat_id
mat_id += 1
bond = params['materials']['bond']
# Bond_SP01 参数：s1,s2,s3=滑移控制点；tau1,tau2,tau3=黏结应力；K=初始刚度；r,c,phi,alpha,beta=曲线形状；参见 OpenSees Wiki。
ops.uniaxialMaterial('Bond_SP01', mat_id, bond['s1'], bond['tau1'], bond['s2'], bond['tau2'], bond['s3'], bond['tau3'], bond['K'], bond['r'], bond['c'], bond['phi'], bond['alpha'], bond['beta'])
bond_mat = mat_id
print('Materials defined:', mat_id)


### FiberSection 与非线性梁柱单元
构建圆形 RC 墩截面，并以 forceBeamColumn 单元承载，实现分布式塑性。

In [ ]:
section_tag = 1
pier = params['section']['pier']
radius = pier['diameter'] / 2.0
cover = pier['cover']
core_radius = radius - cover
num_core_fibers = 32
num_cover_fibers = 16
long_bars = pier['longitudinal_bars']
bar_area = pier['bar_area']

ops.section('Fiber', section_tag)
ops.patch('circ', cover_mat, num_cover_fibers, 4, 0.0, 0.0, core_radius, radius)
ops.patch('circ', core_mat, num_core_fibers, 8, 0.0, 0.0, 0.0, core_radius)
ops.layer('circ', steel_mat, long_bars, bar_area, 0.0, 0.0, core_radius)

gtransf_tag = 1
ops.geomTransf('PDelta', gtransf_tag, 0.0, 0.0, 1.0)

column_tag = 1000
column_elems = []
for i in range(len(pier_top_tags) - 1):
    base_tag = 200 + i
    top_tag = pier_top_tags[i + 1]
    ops.node(base_tag, pier_coords[i + 1][0], pier_coords[i + 1][1], pier_coords[i + 1][2])
    ops.fix(base_tag, 1, 1, 1, 1, 1, 1)
    ops.element('forceBeamColumn', column_tag, base_tag, top_tag, gtransf_tag, section_tag)
    column_elems.append(column_tag)
    column_tag += 1

print('Column elements:', column_elems)


### 截面纤维可视化
使用 opsvis 绘制纤维截面核对钢筋与混凝土布置。

In [ ]:
opsv.plot_fiber_section(None, section_tag, npts=200)


### 柱端零长度截面单元与 Bond_SP01
在墩柱端面复制节点并插入零长度截面单元以模拟粘结滑移。

In [ ]:
zero_length_tags = []
for i, top_tag in enumerate(pier_top_tags[1:-1], start=1):
    dup_tag = top_tag + 500
    ops.node(dup_tag, *ops.nodeCoord(top_tag))
    ops.equalDOF(top_tag, dup_tag, 1, 2, 3)
    zl_tag = 3000 + i
    ops.element('zeroLengthSection', zl_tag, top_tag, dup_tag, section_tag)
    zero_length_tags.append(zl_tag)
print('Zero-length section elements:', zero_length_tags)


### 基础柔度建模
以零长度单元叠加平动与转动弹簧表示基础柔度。

In [ ]:
foundation_elements = []
ktx, kty, ktz = params['foundation']['translational_stiffness']
kxr, kyr, kzr = params['foundation']['rotational_stiffness']
for i, pier_tag in enumerate(pier_top_tags[1:-1], start=1):
    base_tag = 200 + i
    spring_node = base_tag + 1000
    ops.node(spring_node, *ops.nodeCoord(base_tag))
    ops.fix(spring_node, 1, 1, 1, 1, 1, 1)
    ops.uniaxialMaterial('Elastic', 4000 + i, ktx)
    ops.uniaxialMaterial('Elastic', 4100 + i, kty)
    ops.uniaxialMaterial('Elastic', 4200 + i, ktz)
    ops.uniaxialMaterial('Elastic', 4300 + i, kxr)
    ops.uniaxialMaterial('Elastic', 4400 + i, kyr)
    ops.uniaxialMaterial('Elastic', 4500 + i, kzr)
    zl_tag = 5000 + i
    ops.element('zeroLength', zl_tag, base_tag, spring_node, '-mat', 4000 + i, 4100 + i, 4200 + i, 4300 + i, 4400 + i, 4500 + i, '-dir', 1, 2, 3, 4, 5, 6)
    foundation_elements.append(zl_tag)
print('Foundation springs:', foundation_elements)


### 上部结构：桥面梁单元与质量
使用 elasticBeamColumn 连通主节点，并赋予集中质量。

In [ ]:
Edeck = 3.4e7
Adeck = params['section']['deck']['width'] * params['section']['deck']['flange_thickness']
Iz = 200.0
Iy = 150.0
J = 80.0
beam_elems = []
beam_tag = 6000
for nd1, nd2 in zip(node_tags[:-1], node_tags[1:]):
    ops.element('elasticBeamColumn', beam_tag, nd1, nd2, Adeck, Edeck, Edeck/2.6, J, Iy, Iz, gtransf_tag)
    beam_elems.append(beam_tag)
    beam_tag += 1
    mass = 0.8
    ops.mass(nd1, mass, mass, mass, 0.0, 0.0, mass*0.1)
ops.mass(node_tags[-1], 0.8, 0.8, 0.8, 0.0, 0.0, 0.08)
print('Beam elements created:', len(beam_elems))


### 刚性连接桥面与墩顶
通过 rigidLink beam 实现桥面横向刚臂与墩顶刚结。

In [ ]:
rigid_links = []
for top_tag in pier_top_tags[1:-1]:
    nearest = min(node_tags, key=lambda tag: abs(ops.nodeCoord(tag)[0] - ops.nodeCoord(top_tag)[0]))
    ops.rigidLink('beam', top_tag, nearest)
    rigid_links.append((top_tag, nearest))
print('Rigid links:', rigid_links)


### 台后回填土函数
定义函数生成非线性弹簧，若缺少 Hyperbolic Gap Softening，则以 Hysteretic 材料替代。

In [ ]:
def create_abutment_backfill(node_tag, params):
    cfg = params['abutment']
    # Hysteretic 材料三折线骨架：正向与负向对称
    mat_tag = 7000 + node_tag
    k = cfg['backfill_k']
    uy = cfg['backfill_uy']
    res = cfg['residual_ratio'] * k
    ops.uniaxialMaterial('Hysteretic', mat_tag,
                         k*uy, k, res,
                         -k*uy, -k, -res,
                         0.0, 0.0, 0.0,
                         0.0, 0.0, 0.0)
    spring_node = node_tag + 9000
    ops.node(spring_node, *ops.nodeCoord(node_tag))
    ops.fix(spring_node, 1,1,1,1,1,1)
    elem_tag = 8000 + node_tag
    ops.element('zeroLength', elem_tag, node_tag, spring_node, '-mat', mat_tag, '-dir', 1)
    return {"material": mat_tag, "element": elem_tag}


### 桥台基础函数
生成双线性或弹塑性间隙弹簧，表示桥台基础反应。

In [ ]:
def create_abutment_foundation(node_tag, params):
    mat_tag = 8100 + node_tag
    ops.uniaxialMaterial('ElasticPPGap', mat_tag, params['foundation']['translational_stiffness'][0], params['abutment']['backfill_uy'], 0.0)
    spring_node = node_tag + 9100
    ops.node(spring_node, *ops.nodeCoord(node_tag))
    ops.fix(spring_node, 1,1,1,1,1,1)
    elem_tag = 8200 + node_tag
    ops.element('zeroLength', elem_tag, node_tag, spring_node, '-mat', mat_tag, '-dir', 1)
    return {"material": mat_tag, "element": elem_tag}


### 支座函数
以零长度单元承载双线性滞回，模拟橡胶支座剪切弹簧。

In [ ]:
def create_bearing(node_i, node_j, params):
    cfg = params['bearing']
    mat_tag = 8300 + node_i + node_j
    uy = cfg['yield_force'] / cfg['ke']
    ops.uniaxialMaterial('ElasticPP', mat_tag, cfg['ke'], uy)
    elem_tag = 8400 + node_i + node_j
    ops.element('zeroLength', elem_tag, node_i, node_j, '-mat', mat_tag, '-dir', 2)
    return {"material": mat_tag, "element": elem_tag}


### 剪力键函数
使用 Hysteretic 材料三折线骨架表示剪力键力—位移关系。

In [ ]:
def create_shear_key(node_i, node_j, params):
    cfg = params['shear_key']
    mat_tag = 8500 + node_i + node_j
    ops.uniaxialMaterial('Hysteretic', mat_tag,
                         cfg['fy1'], cfg['k1'], cfg['residual'],
                         -cfg['fy1'], -cfg['k1'], -cfg['residual'],
                         cfg['fy2'], cfg['k2'], cfg['k3'],
                         -cfg['fy2'], -cfg['k2'], -cfg['k3'])
    elem_tag = 8600 + node_i + node_j
    ops.element('zeroLength', elem_tag, node_i, node_j, '-mat', mat_tag, '-dir', 3)
    return {"material": mat_tag, "element": elem_tag}


### 碰撞缝函数
调用 ImpactMaterial 模拟赫兹接触的压向间隙。

In [ ]:
def create_impact_gap(node_i, node_j, params):
    cfg = params['impact']
    mat_tag = 8700 + node_i + node_j
    ops.uniaxialMaterial('ImpactMaterial', mat_tag, cfg['gap'], cfg['k1'], cfg['k2'], cfg['mu'])
    elem_tag = 8800 + node_i + node_j
    ops.element('zeroLength', elem_tag, node_i, node_j, '-mat', mat_tag, '-dir', 1)
    return {"material": mat_tag, "element": elem_tag}


### 整桥组装与可视化核查
调用前述部件函数，组装台后回填土、基础弹簧、支座与剪力键，并绘制模型。

In [ ]:
component_registry = {
    "backfills": [],
    "foundations": [],
    "bearings": [],
    "shear_keys": [],
    "impacts": []
}

left_abutment = node_tags[0]
right_abutment = node_tags[-1]
component_registry['backfills'].append(create_abutment_backfill(left_abutment, params))
component_registry['backfills'].append(create_abutment_backfill(right_abutment, params))
component_registry['foundations'].append(create_abutment_foundation(left_abutment, params))
component_registry['foundations'].append(create_abutment_foundation(right_abutment, params))

for top_tag in pier_top_tags[1:-1]:
    nearest = min(node_tags, key=lambda tag: abs(ops.nodeCoord(tag)[0] - ops.nodeCoord(top_tag)[0]))
    component_registry['bearings'].append(create_bearing(top_tag, nearest, params))
    component_registry['shear_keys'].append(create_shear_key(top_tag, nearest, params))
    component_registry['impacts'].append(create_impact_gap(top_tag, nearest, params))

opsv.plot_model()
component_registry


### 重力工况与初始平衡
施加自重载荷并进行静力分析以获得初始状态。

In [ ]:
ops.timeSeries('Linear', 1)
ops.pattern('Plain', 1, 1)
for nd in node_tags:
    ops.load(nd, 0.0, 0.0, -9.8*0.8, 0.0, 0.0, 0.0)
ops.system('BandGeneral')
ops.numberer('RCM')
ops.constraints('Plain')
ops.integrator('LoadControl', 1.0/params['analysis']['gravity_steps'])
ops.algorithm('Newton')
ops.analysis('Static')
for _ in range(params['analysis']['gravity_steps']):
    ops.analyze(1)
print('Gravity analysis completed')


### 地震输入与记录器
采用 timeSeries Path 施加地震动，并配置记录器捕捉关键响应。

In [ ]:
import os
if not os.path.exists('input'):
    os.makedirs('input')
if not os.path.exists(params['analysis']['time_series_file']):
    with open(params['analysis']['time_series_file'], 'w') as f:
        for _ in range(int(params['analysis']['total_time']/params['analysis']['dt'])):
            f.write(f"{0.0}
")

if not os.path.exists('output'):
    os.makedirs('output')

ts_tag = 2
ops.timeSeries('Path', ts_tag, '-dt', params['analysis']['dt'], '-filePath', params['analysis']['time_series_file'])
ops.pattern('UniformExcitation', 2, 1, '-accel', ts_tag)

ops.recorder('Node', '-file', 'output/deck_disp.out', '-time', '-node', *node_tags, '-dof', 1, 2, 3, 'disp')
ops.recorder('Element', '-file', 'output/column_force.out', '-time', '-ele', *column_elems, 'force')


### 动力分析与振型可视化
完成模态分析、地震时程积分，并展示振型与变形形态。

In [ ]:
ops.wipeAnalysis()
ops.system('BandGeneral')
ops.numberer('RCM')
ops.constraints('Plain')
ops.algorithm('Newton')
ops.integrator('Newmark', 0.5, 0.25)
ops.analysis('Transient')
num_steps = int(params['analysis']['total_time']/params['analysis']['dt'])
for _ in range(num_steps):
    ops.analyze(1, params['analysis']['dt'])

ops.wipeAnalysis()
ops.system('BandGeneral')
ops.numberer('RCM')
ops.constraints('Plain')
ops.algorithm('Linear')
ops.integrator('LoadControl', 0.0)
ops.analysis('Transient')
ops.eigen(2)
opsv.plot_mode_shape(1)
opsv.plot_defo(node_tags, az=30, el=15)
